In [1]:
from folium.plugins import MiniMap
import pandas as pd
import numpy as np
from glob import glob
import requests
import folium
import os

In [2]:
# radius와 lat,lon를 이용해서 키워드 검색. No return
def detect_by_radius(keyword, bd_list, lat, lon, radius):
    page_num = 1

    while(1):
        # limitation on page_num is 3
        if page_num == 4:
            break
            
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        params = {'query': keyword,'page': page_num, 'x': lon, 'y' : lat ,'radius' : radius}
        headers = {"Authorization": "KakaoAK a27d09df89174b2ae5990dd9a56366ef"}
        
        resp = requests.get(url, params = params, headers = headers)
        results = resp.json()['documents']     
        
        for result in results:
            result['keyword'] = keyword
        
        if resp.json()['meta']['is_end']:
            bd_list.extend(results)
            break
        else:
            #print('-next page')
            page_num += 1
            bd_list.extend(results)
            

# data없을시 none 입력
def append_col(bd_list, col):
    try:
        bd_list.append(col)
    except:
        bd_list.append("none")

In [3]:
# South Korea Search Points Extraction

CSV_PATH = r"C:\Users\mk156\Desktop\old\classfication\data\axis\csv"
os.chdir(CSV_PATH)
csv_list = glob("*.csv")
pt_list = []
rad = 10

for csv in csv_list:
    df_csv = pd.read_csv(csv, index_col=None)
    print("* " + csv + " Loaded.")
    
    for id in df_csv["OID"]:
        if ((int(id) % (2*rad)) == rad) and ((int(id) % (rad*600)) < 300):
            lat = df_csv["lat_e"][int(id)-1]
            lon = df_csv["lon_e"][int(id)-1] 
            pt_list.append([id, lat, lon])

        if ((int(id) % (2*rad)) == 0) and ((int(id) % (rad*300)) < 300) and ((int(id) / (rad*300)) >= 1):
            lat = df_csv["lat_e"][int(id)-1] 
            lon = df_csv["lon_e"][int(id)-1] 
            pt_list.append([id, lat, lon])

print("*** Points selected: " + str(len(pt_list)))

* bottom150x300.csv Loaded.
* down150x300.csv Loaded.
* mid150x300.csv Loaded.
* up150x300.csv Loaded.
*** Points selected: 1324


In [4]:
# Search PLACES #
attributes = ['bd_name', 'bd_address', 'bd_id', 'bd_lat', 'bd_lon', 'cat_gr', 'cat_cd', 'cat_name']
main_df = pd.DataFrame(columns = attributes)

# PARAMETER - 키워드 LIST로 입력
count = 0
gap = 0.025
bd_list = []
radius = 10000
keyword_list = ["고등학교", "대학교"]

for index, point in enumerate(pt_list):
    page_num = 1
    lat, lon = point[1], point[2]

    #print("--------------------------------------")
    for keyword in keyword_list:
        
        while(1):
            url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
            params = {'query': keyword,'page': page_num, 'x': lon, 'y' : lat ,'radius' : radius}
            headers = {"Authorization": "KakaoAK a27d09df89174b2ae5990dd9a56366ef"}
            resp = requests.get(url, params = params, headers = headers)
            results = resp.json()['documents']
            
            for result in results:
                result['keyword'] = keyword
            
            search_count = resp.json()['meta']['total_count']
            
            if index % 100 == 0:
                print('* %d Building detected for %s : %d' % (index, keyword, search_count))

            if search_count > 45:
                #print('4등분으로 다시 계산.')
                detect_by_radius(keyword, bd_list, lat + 0.025, lon, radius*0.6)
                detect_by_radius(keyword, bd_list, lat, lon + 0.025, radius*0.6)
                detect_by_radius(keyword, bd_list, lat - 0.025, lon, radius*0.6)
                detect_by_radius(keyword, bd_list, lat, lon - 0.025, radius*0.6)
                break

            else:
                if resp.json()['meta']['is_end']:
                    bd_list.extend(results)

                    break
                else:                
                    #print('다음페이지')
                    page_num += 1
                    bd_list.extend(results)
            
print("*** DONE *** DONE *** DONE ***")

* 0 Building detected for 고등학교 : 0
* 0 Building detected for 대학교 : 0
* 100 Building detected for 고등학교 : 0
* 100 Building detected for 대학교 : 0
* 200 Building detected for 고등학교 : 0
* 200 Building detected for 대학교 : 3
* 300 Building detected for 고등학교 : 18
* 300 Building detected for 고등학교 : 18
* 300 Building detected for 대학교 : 24
* 400 Building detected for 고등학교 : 8
* 400 Building detected for 대학교 : 0
* 500 Building detected for 고등학교 : 16
* 500 Building detected for 고등학교 : 16
* 500 Building detected for 대학교 : 15
* 600 Building detected for 고등학교 : 10
* 600 Building detected for 대학교 : 32
* 600 Building detected for 대학교 : 32
* 600 Building detected for 대학교 : 32
* 700 Building detected for 고등학교 : 35
* 700 Building detected for 고등학교 : 35
* 700 Building detected for 고등학교 : 35
* 700 Building detected for 대학교 : 197
* 800 Building detected for 고등학교 : 4
* 800 Building detected for 대학교 : 15
* 900 Building detected for 고등학교 : 0
* 900 Building detected for 대학교 : 0
* 1000 Building detected for 고등학교 : 9


In [5]:
# DATA -> DATAFRAME -> CSV
(bd_name, bd_address, bd_id) = ([],[],[])
(bd_lat, bd_lon, key_list) = ([],[],[])
(cat_gr, cat_cd, cat_name) = ([],[],[])

for bd in bd_list:
    attributes = ['bd_name', 'bd_address', 'bd_id', 'bd_lat', 'bd_lon', 'cat_gr', 'cat_cd', 'cat_name', 'keyword']

    # 불발이 뜰수 있어서 키에러를 잡아줘야 한다...
    append_col(bd_name, bd['place_name'])
    append_col(bd_address, bd['address_name'])
    append_col(bd_id, bd['id'])
    append_col(bd_lat, bd['y'])
    append_col(bd_lon, bd['x'])
    append_col(cat_gr, bd['category_group_name'])
    append_col(cat_cd, bd['category_group_code'])
    append_col(cat_name, bd['category_name'])
    append_col(key_list, bd['keyword'])
    
df_dict = {'bd_name':bd_name, 'bd_address':bd_address, 
           'bd_id' : bd_id, 'bd_lat':bd_lat, 'bd_lon':bd_lon, 
           'cat_gr' : cat_gr, 'cat_cd' : cat_cd, 'cat_name' : cat_name, 
           'keyword' : key_list}

# Generate Final DATAFRAME + Duplicates removed.
bd_df = pd.DataFrame(df_dict)
print("Originally: ",len(bd_df))
bd_df.drop_duplicates(['bd_id'], inplace = True)
print("*After duplication removed:  ", len(bd_list))

OUTPUT_PATH = r"C:\Users\mk156\Desktop\old\classfication\data\bd_csv"
bd_df.to_csv(os.path.join(OUTPUT_PATH, "학교_2.csv"), index=False)

Originally:  58829
*After duplication removed:   58829


In [6]:
print(len(bd_list))

58829


In [7]:
bd_list[0:10]

[{'address_name': '전남 해남군 송지면 미야리 821',
  'category_group_code': 'SC4',
  'category_group_name': '학교',
  'category_name': '교육,학문 > 학교 > 고등학교',
  'distance': '1318',
  'id': '7954243',
  'phone': '061-533-2295',
  'place_name': '송지고등학교',
  'place_url': 'http://place.map.kakao.com/7954243',
  'road_address_name': '전남 해남군 송지면 미야길 158-16',
  'x': '126.516152354192',
  'y': '34.3843978879553',
  'keyword': '고등학교'},
 {'address_name': '전남 해남군 송지면 미야리 803',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '교육,학문 > 학교부속시설',
  'distance': '1317',
  'id': '938530494',
  'phone': '061-533-2295',
  'place_name': '송지고등학교 교무실',
  'place_url': 'http://place.map.kakao.com/938530494',
  'road_address_name': '전남 해남군 송지면 미야길 158-16',
  'x': '126.516171853226',
  'y': '34.3844105855331',
  'keyword': '고등학교'},
 {'address_name': '전남 해남군 송지면 미야리 821',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '교육,학문 > 학교부속시설',
  'distance': '1316',
  'id': '124946